# Get connection information for context menu commands

Many extensions want to launch from the context menu. This is what we do for our own features - _New Query_, _New Notebook_ and more! When you run from here, you will be able to tell exactly what connection your ran from.

Let's add our connection command to the context menu for a Server and a Database connection, and see what's different between the two.



In [13]:
cd './2-contextmenus'

In [0]:
!code .

## Menus

You already have a command. Adding it to a menu is simple - just add a menu contribution!

### Just servers
```json
"menus": {
    "objectExplorer/item/context": [
        {
            "command": "extension.showCurrentConnection",
            "when": "nodeType == Server",
            "group": "myextension@1"
        }
    ]
}
```

### Servers and databases
```json
"menus": {
    "objectExplorer/item/context": [
        {
            "command": "extension.showCurrentConnection",
            "when": "nodeType == =~ /^(Server|Database)$/",
            "group": "myextension@1"
        }
    ]
}
```

### Disabling command in command palette

Some actions only make sense from a connected resource. If you need to disable one from showing globally, just add a `"when":"false"` statement for command palette.

```json
"menus": {
    "commandPalette": [
        {
            // This disables the command. Unless "when" is false, all commands show in the command palette
            "command": "extension.showCurrentConnection",
            "when": "false"
        },
    ]
}
```

# Using the context object to get connection info

Like many of our APIs, the context menu options are in a proposed state. This means we _might_ change them, but if we do we will notify published extension authors to avoid breaking them. 

We are actively working on Connection APIs to simplify usage. Examples are only having 1 connection object, making APIs much simpler to use, and more. Expect these to come in the next few months using the `azdata` module.

Here's right-click context API:  

```ts
/**
 * Context object passed as an argument to command callbacks.
 * Defines the key properties required to identify a node in the object
 * explorer tree and take action against it.
 */
export interface ObjectExplorerContext {

    /**
     * The connection information for the selected object.
     * Note that the connection is not guaranteed to be in a connected
     * state on click.
     */
    connectionProfile: IConnectionProfile;
    /**
     * Defines whether this is a Connection-level object.
     * If not, the object is expected to be a child object underneath
     * one of the connections.
     */
    isConnectionNode: boolean;
    /**
     * Node info for objects below a specific connection. This
     * may be null for a Connection-level object
     */
    nodeInfo: NodeInfo;
}
```

## Adding proposed APIs to your extension
* Will have option by default in future extension templates
* For now, copy contents of https://raw.githubusercontent.com/Microsoft/azuredatastudio/master/src/sql/sqlops.proposed.d.ts to `src/typings/sqlops.proposed.d.ts

## Using right-click menu context

* Convert the method to async (it's just easier when dealing with branching logic)
* define an optional parameter for the context
* Read connection info from one of the 2 launch points

```ts
context.subscriptions.push(vscode.commands.registerCommand('extension.showCurrentConnection',
async (context?: sqlops.ObjectExplorerContext) => 
{
    if (context && context.connectionProfile) {
        // We launched from the Connections viewlet's context menu
        let serverName = context.connectionProfile.serverName;
        let dbName = context.connectionProfile.databaseName || '<Default>';
        vscode.window.showInformationMessage('Using ' + serverName + ':' + dbName);
    } else {
        // We launched from the command palette
        // Note: in the future all APIs will return IConnectionProfile!
        let connection: sqlops.connection.Connection = await sqlops.connection.getCurrentConnection();
        let serverName = connection.options['server'];
        let dbName = connection.options['database'] || '<Default>';

        let message = connection ? 
            'Using ' + serverName + ':' + dbName
            : 'No Connection found!';
        vscode.window.showInformationMessage(message);
    }
}));

```
